In this short tutorial we will install and use [MARTINI](https://martini.readthedocs.io/en/latest/), an analysis package for creating mock HI-data cubes similar to radio interferometer data, written by Kyle Oman (kyle.a.oman@durham.ac.uk). This example uses the input from the [IllustrisTNG](https://ui.adsabs.harvard.edu/abs/2018MNRAS.473.4077P/abstract) simulations. The data are publicly available and hosted at [tng-project.org](https://www.tng-project.org/).

This tutorial can be run either on the [TNG JupyterLab environment](https://www.tng-project.org/data/lab/), or on any system with internet access.

![MARTINI](https://github.com/kyleaoman/martini/raw/main/martini_banner.png)

MARTINI is a modular package for the creation of synthetic resolved HI line observations (data cubes) of smoothed-particle hydrodynamics simulations of galaxies. The various aspects of the mock-observing process are divided logically into sub-modules handling the data cube, source, beam, noise, spectral model and SPH kernel. MARTINI is object-oriented: each sub-module provides a class (or classes) which can be configured as desired. For most sub-modules, base classes are provided to allow for straightforward customization. Instances of each sub-module class are given as parameters to the Martini class; a mock observation is then constructed by calling a handful of functions to execute the desired steps in the mock-observing process.

The MARTINI documentation is [hosted on ReadTheDocs](https://martini.readthedocs.io/en/latest/).

## Installation

MARTINI requires `python3` version `3.7` or higher.

The following command will use `pip` to download and install [MARTINI from pypi](https://pypi.org/project/astromartini/):

In [1]:
import sys

#!{sys.executable} -m pip install astromartini[tngsource]==2.0.1
!{sys.executable} -m pip install /home/koman/code/martini

Processing /home/koman/code/martini
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for astromartini: filename=astromartini-2.0.1-py3-none-any.whl size=73303 sha256=b2ec0975b1f06269a28a77a1bbd47f0fd1e018acd91f15f2805e9759c5f1e9ae
  Stored in directory: /tmp/pip-ephem-wheel-cache-9h0oosns/wheels/ad/39/16/b54749e0fb41d3bcf02c04c583e493a4f51a3272aabff0b0ed
Successfully built astromartini
  Attempting uninstall: astromartini
    Found existing installation: astromartini 2.0.1
    Uninstalling astromartini-2.0.1:
      Successfully uninstalled astromartini-2.0.1


If you do not have superuser permissions or use a virtual environment, you may wish to add the --user flag.
With this command required dependencies will be fetched and installed automatically. Watch for error messages during installation. For greater control you may also install the dependencies by hand. These are: numpy, astropy, scipy, h5py, six and requests.

We'll also install matplotlib, used in this notebook:

In [2]:
!{sys.executable} -m pip install matplotlib

Let's check that we can `import martini`:

In [3]:
import martini

If this produces errors, you may need to restart the Python kernel of this notebook so that it sees the recently installed packages (Kernel -> Restart in the menubar).

We can run MARTINI's built-in demo to check that all of the basic functionality works:

In [ ]:
from martini import demo

demo()

Source module contained 500 particles with total HI mass of 5.00e+09 solMass.
Pruned particles that will not contribute to data cube, 500 particles remaining with total HI mass of 5.00e+09 solMass.
Inserting source in cube.


 28%|██████████▎                          | 6603/23716 [00:10<00:28, 596.79it/s]

When run successfully, this will make a mock observation of a very simple analytic disc model and write some output to the working directory. Rather than inspect this toy model, let's look at a "real" simulation...

## TNG Data

The `TNGSource` module in MARTINI is designed to run on the IllustrisTNG [JupyterLab](https://www.tng-project.org/data/lab/), or in a standalone mode. If you are running on the TNG JupyterLab then the simulations are stored locally on disk and will be detected and used. Otherwise, the `TNGSource` module will use the TNG [web API](https://www.tng-project.org/data/docs/api/) to download the particles for the galaxy of interest. If running in standalone mode, an API key for the TNG web API is required. You must first [register](https://www.tng-project.org/users/register/) for an account. Once you are registered, your API key can be found [here](https://www.tng-project.org/users/profile/). Enter your API key using the following cell (if you are running this notebook on the TNG JupyterLab, you may leave it blank):

In [ ]:
from getpass import getpass

api_key = getpass("TNG web API key:")

We can choose a galaxy of interest by selection a [simulation](https://www.tng-project.org/data/docs/background/), a [snapshot](http://www.tng-project.org/data/docs/specifications/#sec1a) and a subhalo ID. One way to search for a subhalo ID is to use [this tool](https://www.tng-project.org/data/search/) - the `ID` column in search results contains subhalo IDs. For example:

In [ ]:
simulation = "TNG100-1"
snapshot = 99
subhalo_id = 400547  # a central subhalo with 218 < Vmax < 220, and SFR > 1

## TNG Example

First, import some modules from MARTINI, and the units module from astropy.

In [ ]:
from martini.sources import TNGSource
from martini import DataCube, Martini
from martini.beams import GaussianBeam
from martini.noise import GaussianNoise
from martini.spectral_models import GaussianSpectrum
from martini.sph_kernels import (
    AdaptiveKernel,
    GaussianKernel,
    CubicSplineKernel,
    DiracDeltaKernel,
)
import astropy.units as U

The different martini sub-modules need to be initialized, see the [full documentation](https://kyleaoman.github.io/martini/build/html/) for details of all configuration options. A few suggested best-practices specific to TNG are outlined below.

### SOURCE

The rotation configuration takes an inclination (here 60deg) and
rotation about the pole (here 90deg, relative to an arbitrary reference direction). The code attempts to
automatically align the galactic disk in the y-z plane by aligning
the angular momentum along the x-axis. The polar rotation is then
applied, and finally the disc inclined by a rotation around the
y-axis (the line of sight is along the x-axis). The automatic
alignment will work for typical reasonably isolated discs, but will
struggle when companions are present, when the angular momentum axis
is a poor tracer of the disc plane, and especially for satellites. If
finer control of the orientation is needed, derive the transformation
from the simulation box coordinates to the desired coordinates for
the 'observation', keeping in mind that the line of sight is along
the x-axis. This rotation matrix can then be passed to as
`rotation={'rotmat': np.eye(3)}` (here the identity rotation matrix used as a place holder). A common problem in this case is deriving the inverse
transform instead of the forward transform, if unexpected results are
obtained, first try passing the transpose of the rotation matrix.

Notice that your API key must be provided unless you are using `TNGSource` on the TNG JupyterLab (if you provide it anyways in that case, it will simply be ignored).

Any downloaded data can be cached using the `cutout_dir` parameter to specify a directory, if a cache for the requested galaxy is found it will be used instead of re-downloading the data. If running on the TNG JupyterLab environment, this parameter is ignored.


In [ ]:
source = TNGSource(
    simulation,
    snapshot,
    subhalo_id,
    api_key=api_key,
    cutout_dir=".",
    distance=4 * U.Mpc,
    rotation={"L_coords": (60 * U.deg, 0.0 * U.deg)},
    ra=0.0 * U.deg,
    dec=0.0 * U.deg,
)

### DATACUBE

It is usually advisable to set the centre of the cube to track the
centre of the source, as illustrated below. Note that the source
systemic velocity is set according to the distance, peculiar velocity, and Hubble's law.
These values can instead be set explicitly, if desired. A datacube
with 128x128 pixels usually takes a few minutes, depending on the number of particles. 1024x1024 can take
several hours. The number of channels has less influence on the
runtime. Most of the runtime is spent when `M.insert_source_in_cube` is
called below.

In [ ]:
datacube = DataCube(
    n_px_x=384,
    n_px_y=384,
    n_channels=50,
    px_size=10.0 * U.arcsec,
    channel_width=16.0 * U.km * U.s**-1,
    velocity_centre=source.vsys,
    ra=source.ra,
    dec=source.dec,
)

### BEAM

It is usually advisable to set the beam size to be ~3x the pixel
size. Note that the data cube is padded according to the size of the
beam, this usually results in the number of pixel rows printed in the
progress messages to differ from the requested dimensions. The
padding is required for accurate convolution with the beam, but
contains incorrect values after convolution and is discarded to
produce the final data cube of the requested size.

In [ ]:
beam = GaussianBeam(
    bmaj=30.0 * U.arcsec, bmin=30.0 * U.arcsec, bpa=0.0 * U.deg, truncate=3.0
)

### NOISE

The noise is normally added before convolution with the beam (as
below in this example). The rms value passed is that corresponding to the desired noise level in the final data cube, in Jy/beam or equivalent units. To obtain consistent random realisations each time the code is run, we provide a random seed (integer).

In [ ]:
noise = GaussianNoise(
    rms=3.0e-8 * U.Jy * U.beam**-1,
    seed=0,
)

### SPECTRAL MODEL

The `thermal` mode estimates the HI line width for each particle based on its properties (temperature, etc.). The 'subgrid' velocity dispersion can also be fixed to a constant value, e.g. `sigma=7 * U.km / U.s`.

In [ ]:
spectral_model = GaussianSpectrum(sigma="thermal")

### SPH KERNEL

Since IllustrisTNG uses a moving mesh hydrodynamics solver (Arepo),
there are no formal SPH smoothing lengths and no specified kernel.
However, approximate smoothing lengths can be derived from the cell
volumes and densities, so a reasonable approximation is to use these for imaging. 

MARTINI's kernel module implementations use approximations which can break down when the particle smoothing lengths are small compared to the size of the pixels (at the distance of the source). If the approximation in question is not accurate enough, MARTINI will report an error. In this case it might be appropriate to choose another SPH kernel module. Some rules of thumb for making this choice:
 - For very low resolution observations (distant source and few pixels), use the `DiracDeltaKernel`.
 - For high resolution observations (nearby source and many pixels), preferably use the kernel matching that used to run the simulation. In the case of Arepo, which uses a moving mesh rather than SPH, the `CubicSplineKernel` is suggested.
 - The `GaussianKernel` with `truncate=6` has the least stringent requirements for accuracy, but is significantly slower because of its large extent and so should not be used as a first choice.
 - For intermediate resolution observations (when the kernel size is similar to the pixel size), it may be difficult to find a kernel which does not raise an error. In this case, you may consider using the AdaptiveKernel. This module accepts a list of other kernels in order of decreasing priority. The first which is sufficiently accurate is used on a per-particle basis. This usage is illustrated in the example below - first the `CubicSplineKernel` is tried for particles with long smoothing lengths relative to the pixel scale, then the `DiracDeltaKernel` is used for particles with small smoothing lengths relative to the pixels, and finally falling back on the `GaussianKernel` for intermediate smoothing lengths.

In [ ]:
sph_kernel = AdaptiveKernel(
    (CubicSplineKernel(), DiracDeltaKernel(), GaussianKernel(truncate=6))
)

### MARTINI

Now set up the configuration, and do the actual run:

In [ ]:
M = Martini(
    source=source,
    datacube=datacube,
    beam=beam,
    noise=noise,
    spectral_model=spectral_model,
    sph_kernel=sph_kernel,
)
M.insert_source_in_cube()
M.add_noise()
M.convolve_beam()

You may notice that the number of pixels in the progress counter differs from the number defined in the DataCube module. This is because convolution with the beam requires some padding, which is ultimately filled with nonsense and discarded.

To write the results: two output formats are available, depending on preference. Both
formats are self-documenting, via FITS header keywords and HDF5
attributes, respectively. For HDF5 output, the beam image is included
in the same file. (If you do not have h5py installed, comment out the call to `write_hdf5`.)

In [ ]:
M.write_fits("tngdemo.fits", channels="velocity")
M.write_beam_fits("tngdemo_beam.fits", channels="velocity")
M.write_hdf5("tngdemo.hdf5", channels="velocity")

### Inspect the results

Let's load the HDF5 that MARTINI produced and take a quick look.

In [ ]:
import h5py

f = h5py.File("tngdemo.hdf5", "r")

In [ ]:
list(f.keys())

In [ ]:
FluxCube = f["FluxCube"][()]
vch = f["channel_mids"][()] / 1e3 - source.distance.to(U.Mpc).value * 70  # m/s to km/s
f.close()

In [ ]:
FluxCube.shape

Let's examine one of the velocity channels:

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig = plt.figure(figsize=(5, 5))
ax = fig.add_subplot(1, 1, 1)

plt.imshow(FluxCube[:, :, 15].T, cmap="Greys", aspect=1.0, origin="lower")
ax.autoscale(False)
ax.set_xlabel("x [px = arcsec/{:.0f}]".format(datacube.px_size.to(U.arcsec).value))
ax.set_ylabel("y [px = arcsec/{:.0f}]".format(datacube.px_size.to(U.arcsec).value))
plt.colorbar(label="Flux [Jy/beam]");

And do a quick plot of the first three moments:

In [ ]:
import numpy as np

np.seterr(all="ignore")
fig = plt.figure(figsize=(16, 5))
sp1 = fig.add_subplot(1, 3, 1)
sp2 = fig.add_subplot(1, 3, 2)
sp3 = fig.add_subplot(1, 3, 3)
rms = np.std(FluxCube[:16, :16])  # noise in a corner patch where there is little signal
clip = np.where(FluxCube > 5 * rms, 1, 0)
mom0 = np.sum(FluxCube, axis=-1)
mask = np.where(mom0 > 0.05, 1, np.nan)
mom1 = np.sum(FluxCube * clip * vch, axis=-1) / mom0
mom2 = (
    np.sqrt(np.sum(FluxCube * clip * np.power(vch - mom1[..., np.newaxis], 2), axis=-1))
    / mom0
)
im1 = sp1.imshow(mom0.T, cmap="Greys", aspect=1.0, origin="lower")
plt.colorbar(im1, ax=sp1, label="mom0 [Jy/beam]")
im2 = sp2.imshow((mom1 * mask).T, cmap="RdBu", aspect=1.0, origin="lower")
plt.colorbar(im2, ax=sp2, label="mom1 [km/s]")
im3 = sp3.imshow(
    (mom2 * mask).T, cmap="magma", aspect=1.0, origin="lower", vmin=0, vmax=300
)
plt.colorbar(im3, ax=sp3, label="mom2 [km/s]")
for sp in sp1, sp2, sp3:
    sp.set_xlabel("x [px = arcsec/10]")
    sp.set_ylabel("y [px = arcsec/10]")
plt.subplots_adjust(wspace=0.3)

This galaxy clearly has a very nice spiral morphology in HI, has a rotation-dominated velocity field, and is interacting with a companion. The alignment of the disc looks as expected - the inclination looks to be about 60 degrees, and the position angle is horizontal in the figure - in this case the automated orientation function has performed well, though it should never be assumed that this will always be the case!

For complete documentation, more usage examples, and further information, please take a look at the [MARTINI webpage](https://kyleaoman.github.io/martini).